In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import pandas as pd
import time

Setting up the webdriver
Plus, running the browser headless

In [2]:
chrome_options = Options()
chrome_options.add_argument("--headless")
driver = webdriver.Chrome("/Users/asadimam/ChromeDriver/chromedriver-2",options=chrome_options)



In [3]:
# This is where we initialise all the necessary variables and lists    
# These are empty lists
size = [] 
desc = []
sale = []
location = []
property_type = []

# These are the necessary variables to run the loop
prop_type = {
            'Offices':1,
            'Industrial':2,
            'Land':3,
            'Retail':4,
            'Leisure':5,
            'Other':6
                }
prop_type_list = list(prop_type.values())
prop_type_list_string = list(map(str,prop_type_list))
locations = ['reading','slough','maidenhead','bristol','swindon']

In [4]:
# This is the scraping code

for t, x in [(t,x) for t in prop_type_list_string for x in locations]:
        p= 1
        while True:
            url = 'https://www.novaloca.com/property-search-results/default.aspx?page={}&propertytypes={}&saletypes=1&status=1&sizefrom=1&sizeto=500000%2b&sizetype=2&location={}&distance=30&addresskeyword=&excludeServicedOffices=False'.format(p,t,x)
            driver.get(url)
            time.sleep(3)
            p += 1
            try:
                # Test to see if we're on the last page
                test = driver.find_element_by_class_name('extra-details')

#                 print(f'this is {p} of {x}')

                # Execution Code For Every Page
                ids = driver.find_elements_by_xpath("//*[contains(@id,'property-')]")
                property_ids = []
                for i in ids:
                    property_ids.append(i.get_attribute('id'))
                    for j in property_ids:
                        #     To extract sizes from each property-card
                        size_test = driver.find_elements_by_xpath('//*[@id="'+j+'"]/div/div[1]/div[3]/div/div[1]/div/span/strong')[0]    
                        size.append(size_test.text)
                        desc_test = driver.find_elements_by_xpath('//*[@id="'+j+'"]/div/div[2]/h4')[0]
                        desc.append(desc_test.text)
                        sale_test = driver.find_elements_by_xpath('//*[@id="'+j+'"]/div/div[2]/p/span[2]')[0]
                        sale.append(sale_test.text)
                        location.append(x)
                        property_type.append(t)

            except:
                break





In [11]:
df_properties = pd.DataFrame({'city':location,'prop_type':property_type,'prop_size':size,'prop_desc':desc,'prop_price':sale})

In [12]:
df_properties

,city,prop_type,prop_size,prop_desc,prop_price
0,reading,1,"1,340 Sq Ft","3rd Floor Office With Entry Phone System, Thir...","- £340,000.00"
1,reading,1,"1,340 Sq Ft","3rd Floor Office With Entry Phone System, Thir...","- £340,000.00"
2,reading,1,"685 to 2,100 Sq Ft","Unit 1 Cutbush Court, Danehill, Reading, RG6 4UW","- £550,000.00"
3,reading,1,"1,340 Sq Ft","3rd Floor Office With Entry Phone System, Thir...","- £340,000.00"
4,reading,1,"685 to 2,100 Sq Ft","Unit 1 Cutbush Court, Danehill, Reading, RG6 4UW","- £550,000.00"
...,...,...,...,...,...
15789,swindon,6,,FOR SALE BY PUBLIC AUCTION unless previously s...,
15790,swindon,6,,"Olivers Court, Shepherds Spring Lane, Andover,...",
15791,swindon,6,"22,526 Sq Ft","Willow Court, Minns Business Park, Oxford, OX2...","- £8,500,000.00"
15792,swindon,6,82799 Sq Ft,Residential development land with planning per...,


In [17]:
df_new= df_properties.drop_duplicates()

In [22]:
df_new.to_csv('properties-unique.csv',index=False)

In [23]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1342 entries, 0 to 15793
Data columns (total 5 columns):
city          1342 non-null object
prop_type     1342 non-null object
prop_size     1342 non-null object
prop_desc     1342 non-null object
prop_price    1342 non-null object
dtypes: object(5)
memory usage: 62.9+ KB


In [21]:
df_new.isna().sum()

,city,prop_type,prop_size,prop_desc,prop_price
0,False,False,False,False,False
2,False,False,False,False,False
5,False,False,False,False,False
9,False,False,False,False,False
14,False,False,False,False,False
...,...,...,...,...,...
15771,False,False,False,False,False
15775,False,False,False,False,False
15780,False,False,False,False,False
15786,False,False,False,False,False
